In [2]:
import warnings
warnings.filterwarnings('ignore')

In [3]:
import numpy as np
import pandas as pd
import tensorflow as tf

In [4]:
data = pd.read_excel('C:/Users/Aravind/Work/PROJECTS/Milky-way-of-Deep-Learning/Milky-way-of-Deep-Learning/data/external/full_dataset.xlsx')

In [5]:
from sklearn.preprocessing import OrdinalEncoder
encoder = OrdinalEncoder()
data[['Diet']] = encoder.fit_transform(data[['Diet']])

In [6]:
X = data.drop(['Diet'] , axis=1)
y = data['Diet']

In [7]:
from sklearn.model_selection import train_test_split

X_train_full, X_test, y_train_full, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
X_train, X_valid, y_train, y_valid = train_test_split(X_train_full, y_train_full, test_size=0.25, random_state=42)

In [16]:
tf.random.set_seed(42)
model = tf.keras.Sequential([
    tf.keras.layers.LSTM(512, return_sequences=True, input_shape=[None, 1]),
    tf.keras.layers.LSTM(256),
    tf.keras.layers.Dense(3 , activation="softmax")
])

initial_learning_rate = 0.01
lr_schedule = tf.keras.optimizers.schedules.ExponentialDecay(
    initial_learning_rate, decay_steps=10000, decay_rate=0.9)
optimizer = tf.keras.optimizers.Adam(learning_rate=lr_schedule)

model.compile(
    loss="sparse_categorical_crossentropy",
    optimizer=optimizer,
    metrics=["accuracy"]
)
history = model.fit(X_train ,  y_train, epochs=10,
                    validation_data=(X_valid, y_valid))

Epoch 1/10
 2/62 [..............................] - ETA: 10:46 - loss: 2.3081 - accuracy: 0.3125

KeyboardInterrupt: 

In [8]:
import torch
import torch.nn as nn
from torch.nn import functional as F
device = 'cuda' if torch.cuda.is_available() else 'cpu'
print(device)

cuda


In [15]:
import torch.optim as optim
import numpy as np

# Set the random seed for reproducibility
torch.manual_seed(42)

# Define the model
class CustomLSTM(nn.Module):
    def __init__(self, input_size, hidden_size, output_size):
        super(CustomLSTM, self).__init__()
        self.lstm1 = nn.LSTM(input_size, hidden_size, batch_first=True)
        self.lstm2 = nn.LSTM(hidden_size, 1024, batch_first=True)
        self.fc = nn.Linear(512, output_size)
        self.softmax = nn.Softmax(dim=1)

    def forward(self, x):
        out, _ = self.lstm1(x)
        out, _ = self.lstm2(out)
        out = self.fc(out[:, -1, :])
        out = self.softmax(out)
        return out

input_size = 1  # Input features
hidden_size = 1024  # LSTM units
output_size = 3  # Number of output classes

model = CustomLSTM(input_size, hidden_size, output_size)

# Define the learning rate schedule and optimizer
initial_learning_rate = 0.01
lr_lambda = lambda epoch: 0.9 ** (epoch / 10000)
optimizer = optim.Adam(model.parameters(), lr=initial_learning_rate)
scheduler = optim.lr_scheduler.LambdaLR(optimizer, lr_lambda=lr_lambda)

# Define the loss function
criterion = nn.CrossEntropyLoss()

# Convert your data to NumPy arrays and then to PyTorch tensors
X_train_np = np.array(X_train)
y_train_np = np.array(y_train)
X_valid_np = np.array(X_valid)
y_valid_np = np.array(y_valid)

X_train_tensor = torch.Tensor(X_train_np).unsqueeze(2)
y_train_tensor = torch.LongTensor(y_train_np)
X_valid_tensor = torch.Tensor(X_valid_np).unsqueeze(2)
y_valid_tensor = torch.LongTensor(y_valid_np)

# Train the model
epochs = 10
for epoch in range(epochs):
    model.train()
    optimizer.zero_grad()
    outputs = model(X_train_tensor)
    loss = criterion(outputs, y_train_tensor)
    loss.backward()
    optimizer.step()
    scheduler.step()

    # Validation
    model.eval()
    with torch.no_grad():
        val_outputs = model(X_valid_tensor)
        val_loss = criterion(val_outputs, y_valid_tensor)
        _, predicted = torch.max(val_outputs, 1)
        accuracy = (predicted == y_valid_tensor).float().mean()

    print(f"Epoch [{epoch+1}/{epochs}], Loss: {loss.item():.4f}, Val Loss: {val_loss.item():.4f}, Val Accuracy: {accuracy.item():.4f}")


RuntimeError: [enforce fail at ..\c10\core\impl\alloc_cpu.cpp:72] data. DefaultCPUAllocator: not enough memory: you tried to allocate 163813240832 bytes.

In [18]:
tf.random.set_seed(42)
model1 = tf.keras.Sequential()
model1.add(tf.keras.layers.Flatten())
for layer in range(20):
    model1.add(tf.keras.layers.Dense(50, activation="relu",
                                    kernel_initializer="he_normal"))
model1.add(tf.keras.layers.Dense(3, activation="softmax"))

In [19]:
model1.compile(loss="sparse_categorical_crossentropy",
              optimizer=tf.keras.optimizers.SGD(learning_rate=0.01),
              metrics=["accuracy"])

In [20]:
history1 = model1.fit(X_train ,  y_train, epochs=20,
                    validation_data=(X_valid, y_valid))

Epoch 1/20
62/62 [==============================] - 1s 5ms/step - loss: 1.1448 - accuracy: 0.3201 - val_loss: 1.1018 - val_accuracy: 0.3359
Epoch 2/20
62/62 [==============================] - 0s 2ms/step - loss: 1.1018 - accuracy: 0.3089 - val_loss: 1.0976 - val_accuracy: 0.3389
Epoch 3/20
62/62 [==============================] - 0s 2ms/step - loss: 1.1006 - accuracy: 0.3506 - val_loss: 1.1013 - val_accuracy: 0.3603
Epoch 4/20
62/62 [==============================] - 0s 2ms/step - loss: 1.1004 - accuracy: 0.3313 - val_loss: 1.1059 - val_accuracy: 0.3130
Epoch 5/20
62/62 [==============================] - 0s 2ms/step - loss: 1.0998 - accuracy: 0.3369 - val_loss: 1.0987 - val_accuracy: 0.3649
Epoch 6/20
62/62 [==============================] - 0s 2ms/step - loss: 1.1026 - accuracy: 0.3359 - val_loss: 1.0961 - val_accuracy: 0.3664
Epoch 7/20
62/62 [==============================] - 0s 2ms/step - loss: 1.1001 - accuracy: 0.3445 - val_loss: 1.0986 - val_accuracy: 0.3313
Epoch 8/20
62/62 [==

In [24]:
score = model1.evaluate(X_test, y_test)

21/21 [==============================] - 0s 1ms/step - loss: 1.0987 - accuracy: 0.3344
